In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv("./archive/test.csv")
print(df.shape)
df = df.dropna()
print(df.shape)

(1946719, 30)
(1946719, 30)


In [3]:
df.head(10)

,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND,CARRIER_HISTORICAL,DEP_AIRPORT_HIST,DAY_HISTORICAL,DEP_BLOCK_HIST
0,4,6,0,1000-1059,3,3,5,110,Delta Air Lines Inc.,6837,...,Atlanta Municipal,0.00,0.0,0.0,64.0,13.87,0.139558,0.180307,0.177124,0.147773
1,3,2,0,1000-1059,2,2,31,50,Endeavor Air Inc.,14450,...,Bangor International,0.00,0.0,0.0,48.0,12.08,0.155571,0.187883,0.132868,0.156045
2,2,5,0,1000-1059,8,2,37,173,United Air Lines Inc.,16530,...,Los Angeles International,0.00,0.0,0.0,54.0,10.07,0.189496,0.240316,0.236965,0.187484
3,4,6,0,0900-0959,11,2,37,181,Alaska Airlines Inc.,17522,...,Portland International,0.00,0.0,0.0,64.0,7.38,0.132326,0.112840,0.177124,0.135374
4,8,7,0,0900-0959,1,2,27,143,Southwest Airlines Co.,14220,...,Spokane International,0.00,0.0,0.0,104.0,4.92,0.192550,0.123487,0.196519,0.138067
5,6,3,1,0700-0759,6,2,45,143,Southwest Airlines Co.,21878,...,Eppley Airfield,0.00,0.0,0.0,83.0,7.83,0.259775,0.201805,0.237175,0.110788
6,1,2,0,1700-1759,6,3,28,173,United Air Lines Inc.,17725,...,Stapleton International,0.69,0.0,0.0,59.0,8.50,0.181882,0.188723,0.159489,0.229539
7,5,1,0,1000-1059,6,3,38,143,Southwest Airlines Co.,14585,...,Hollywood-Burbank Midpoint,0.00,0.0,0.0,91.0,6.04,0.248620,0.190985,0.193668,0.156042
8,12,7,0,1100-1159,1,3,25,143,Southwest Airlines Co.,13152,...,Will Rogers World,0.00,0.0,0.0,47.0,2.24,0.264038,0.163150,0.226980,0.193888
9,11,2,0,0800-0859,5,1,92,180,Delta Air Lines Inc.,31151,...,NONE,0.00,0.0,0.0,63.0,6.71,0.100882,0.187883,0.153130,0.094317


In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [5]:
df = pd.read_csv("./archive/test.csv")
df = df.dropna()

In [6]:
print(df.columns)

Index(['MONTH', 'DAY_OF_WEEK', 'DEP_DEL15', 'DEP_TIME_BLK', 'DISTANCE_GROUP',
       'SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 'NUMBER_OF_SEATS',
       'CARRIER_NAME', 'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'AVG_MONTHLY_PASS_AIRPORT',
       'AVG_MONTHLY_PASS_AIRLINE', 'FLT_ATTENDANTS_PER_PASS',
       'GROUND_SERV_PER_PASS', 'PLANE_AGE', 'DEPARTING_AIRPORT', 'LATITUDE',
       'LONGITUDE', 'PREVIOUS_AIRPORT', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND',
       'CARRIER_HISTORICAL', 'DEP_AIRPORT_HIST', 'DAY_HISTORICAL',
       'DEP_BLOCK_HIST'],
      dtype='object')


In [7]:
# Exclude leaky variables -- we'll only keep what we think could reasonably be known before flight time
features = ['MONTH', 'DAY_OF_WEEK', 'DISTANCE_GROUP',
       'SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 'NUMBER_OF_SEATS',
       'CARRIER_NAME', 'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'AVG_MONTHLY_PASS_AIRPORT',
       'AVG_MONTHLY_PASS_AIRLINE', 'FLT_ATTENDANTS_PER_PASS',
       'GROUND_SERV_PER_PASS', 'PLANE_AGE', 'DEPARTING_AIRPORT', 'LATITUDE',
       'LONGITUDE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND']
features = pd.get_dummies(df[features])
labels = df['DEP_DEL15']

In [8]:
features.head()

,MONTH,DAY_OF_WEEK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,AIRLINE_AIRPORT_FLIGHTS_MONTH,AVG_MONTHLY_PASS_AIRPORT,...,DEPARTING_AIRPORT_Stapleton International,DEPARTING_AIRPORT_Syracuse Hancock International,DEPARTING_AIRPORT_Tampa International,DEPARTING_AIRPORT_Theodore Francis Green State,DEPARTING_AIRPORT_Truax Field,DEPARTING_AIRPORT_Tucson International,DEPARTING_AIRPORT_Tulsa International,DEPARTING_AIRPORT_Washington Dulles International,DEPARTING_AIRPORT_Will Rogers World,DEPARTING_AIRPORT_William P Hobby
0,4,6,3,3,5,110,6837,81803,180,819756,...,False,False,False,False,False,False,False,False,False,False
1,3,2,2,2,31,50,14450,22191,1956,1208249,...,False,False,False,False,False,False,False,False,False,False
2,2,5,8,2,37,173,16530,43512,4679,2743323,...,True,False,False,False,False,False,False,False,False,False
3,4,6,11,2,37,181,17522,21136,1629,2780593,...,False,False,False,False,False,False,False,False,False,False
4,8,7,1,2,27,143,14220,114987,6116,1903352,...,False,False,False,False,False,False,False,False,False,False


In [9]:
# Normalize features
scalefeatures = ['MONTH', 'DAY_OF_WEEK', 'DISTANCE_GROUP',
       'SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 'NUMBER_OF_SEATS',
       'CARRIER_NAME', 'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'AVG_MONTHLY_PASS_AIRPORT',
       'AVG_MONTHLY_PASS_AIRLINE', 'FLT_ATTENDANTS_PER_PASS',
       'GROUND_SERV_PER_PASS', 'PLANE_AGE', 'DEPARTING_AIRPORT', 'LATITUDE',
       'LONGITUDE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND']
scalefeatures = df[scalefeatures]
scaler = MinMaxScaler()
features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

In [10]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

In [11]:
# Define the PyTorch Dataset
class DelayDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        x = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(self.labels.iloc[idx], dtype=torch.float32)
        return x, y

In [12]:
# Initialize Datasets and DataLoaders
train_dataset = DelayDataset(X_train, y_train)
val_dataset = DelayDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [13]:
# Define MLP model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class MLPModel(nn.Module):
    def __init__(self, input_size):
        super(MLPModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.model(x)

In [15]:
model = MLPModel(input_size=features.shape[1]).to(device)

In [16]:
# Define Loss and Optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/opt/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [17]:
# Training Loop with Validation
epochs = 100
mlp_train_losses, mlp_val_losses = [], []

In [18]:
for epoch in range(epochs):
    model.train()
    batch_train_losses = []
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        predictions = model(x_batch).squeeze()
        loss = loss_fn(predictions, y_batch)
        loss.backward()
        optimizer.step()

        batch_train_losses.append(loss.item())

    avg_train_loss = np.mean(batch_train_losses)
    mlp_train_losses.append(avg_train_loss)

    # Validation loop
    model.eval()
    batch_val_losses = []
    with torch.no_grad():
        for batch in val_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            predictions = model(x_batch).squeeze()
            loss = loss_fn(predictions, y_batch)
            batch_val_losses.append(loss.item())

    avg_val_loss = np.mean(batch_val_losses)
    mlp_val_losses.append(avg_val_loss)

    print(f"Epoch {epoch + 1} | Training Loss: {avg_train_loss:.2f} | Validation Loss: {avg_val_loss:.2f}")

KeyboardInterrupt: 

In [20]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.9 MB/s eta 0:00:0000:0100:01


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Convert training and validation sets to DMatrix format
xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_val = xgb.DMatrix(X_val, label=y_val)

# Parameters for XGBoost
params = {
    "objective": "reg:squarederror",
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42
}

# Dictionary to store evaluation metrics for each round
xgb_eval_result = {}

# Train XGBoost model with evaluation at each round
xgb_model = xgb.train(
    params=params,
    dtrain=xgb_train,
    num_boost_round=100,  # Set number of boosting rounds, similar to epochs
    evals=[(xgb_train, "train"), (xgb_val, "validation")],
    evals_result=xgb_eval_result,
    verbose_eval=False
)

# Extract training and validation losses (RMSE) for each boosting round
xgb_train_losses = xgb_eval_result["train"]["rmse"]
xgb_val_losses = xgb_eval_result["validation"]["rmse"]

for i, (train_loss, val_loss) in enumerate(zip(xgb_train_losses, xgb_val_losses), start=1):
    print(f"Boosting Round {i} | Training RMSE: {train_loss:.2f} | Validation RMSE: {val_loss:.2f}")

In [ ]:
# Adjust epochs to match the minimum length available across all lists
epochs = range(1, min(len(mlp_train_losses), len(xgb_train_losses)) + 1)

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# MLP Loss Plot
ax1.plot(epochs, mlp_train_losses[:len(epochs)], label="MLP Training Loss (MSE)", marker='o')
ax1.plot(epochs, mlp_val_losses[:len(epochs)], label="MLP Validation Loss (MSE)", marker='o')
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss (MSE)")
ax1.set_title("MLP Training and Validation Loss")
ax1.legend()
ax1.grid(True)

# XGBoost Loss Plot
ax2.plot(epochs, xgb_train_losses[:len(epochs)], label="XGBoost Training Loss (RMSE)", marker='x')
ax2.plot(epochs, xgb_val_losses[:len(epochs)], label="XGBoost Validation Loss (RMSE)", marker='x')
ax2.set_xlabel("Boosting Round")
ax2.set_ylabel("Loss (RMSE)")
ax2.set_title("XGBoost Training and Validation Loss")
ax2.legend()
ax2.grid(True)

# Show the combined figure
plt.suptitle("Training and Validation Loss Comparison: MLP vs XGBoost")
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
pip install graphviz

In [ ]:
import xgboost as xgb
import graphviz

# Assuming `xgb_model` is the trained XGBoost model
# Export the first tree in Graphviz format
xgboost_tree = xgb.to_graphviz(xgb_model, num_trees=0)

# Save as PDF for high resolution
xgboost_tree.render(filename="xgboost_tree_visualization", format="png")

# Alternatively, you can save as PNG, but PDF will retain the best quality

In [ ]:
import xgboost as xgb
from xgboost import plot_tree
import matplotlib.pyplot as plt

# Assuming `xgb_model` is the trained XGBoost model

# Create a high-resolution figure
plt.figure(figsize=(40, 30), dpi=1000)  # Increase `figsize` and `dpi` for better resolution
plot_tree(xgb_model, num_trees=0, rankdir='LR')
plt.title("Visualization of Boosted Tree (XGBoost)")

# Save the plot to the working directory as a high-resolution PNG
plt.savefig("xgboost_tree_visualization.png", format='png', bbox_inches='tight')

# Display the plot in the notebook
plt.show()

In [ ]:
pip install torchviz

In [ ]:
import torch
from torchviz import make_dot
from IPython.display import Image

# Generate a sample input tensor
sample_input = torch.randn(1, features.shape[1])  # Replace `features.shape[1]` with the actual input size

# Perform a forward pass to create the computation graph
output = model(sample_input)

# Generate and render the MLP structure as an image
dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render("MLP_Structure")  # This saves the image as "MLP_Structure.png"

# Display the generated image in the notebook
Image(filename="MLP_Structure.png")